In [2]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *
import yfinance as yf
import json

In [7]:
from yahoo_fin import stock_info as si

# Obtener el balance general para Apple (AAPL)
balance_sheet = si.get_balance_sheet('AAPL')

# Mostrar el balance general
print(balance_sheet)


IndexError: list index out of range

In [3]:
msft = yf.Ticker("MSFT")
quar = msft.quarterly_income_stmt
quar

,2024-06-30,2024-03-31,2023-12-31,2023-09-30,2023-06-30,2023-03-31,2022-12-31
Tax Effect Of Unusual Items,-16263853.211009,-33660000.0,-29160000.0,-20700000.0,20855987.382214,NaN,NaN
Tax Rate For Calcs,0.191339,0.18,0.18,0.18,0.187892,NaN,NaN
Normalized EBITDA,34416000000.0,33741000000.0,33556000000.0,31845000000.0,28972000000.0,NaN,NaN
Total Unusual Items,-85000000.0,-187000000.0,-162000000.0,-115000000.0,111000000.0,NaN,NaN
Total Unusual Items Excluding Goodwill,-85000000.0,-187000000.0,-162000000.0,-115000000.0,111000000.0,NaN,NaN
Net Income From Continuing Operation Net Minority Interest,22036000000.0,21939000000.0,21870000000.0,22291000000.0,20081000000.0,NaN,NaN
Reconciled Depreciation,6380000000.0,6027000000.0,5959000000.0,3921000000.0,3874000000.0,NaN,NaN
Reconciled Cost Of Revenue,19684000000.0,18505000000.0,19623000000.0,16302000000.0,16795000000.0,NaN,NaN
EBITDA,34331000000.0,33554000000.0,33394000000.0,31730000000.0,29083000000.0,NaN,NaN
EBIT,27951000000.0,27527000000.0,27435000000.0,27809000000.0,25209000000.0,NaN,NaN


In [3]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [4]:
unique_tickers = bigquery.run_query(
    f"""
        SELECT DISTINCT ticker FROM (
        SELECT ticker FROM sara-carles-keepcoding.bronze.bronze_sp500_tickers
        UNION ALL
        SELECT ticker FROM sara-carles-keepcoding.bronze.bronze_russell_2000_tickers
        UNION ALL
        SELECT ticker FROM sara-carles-keepcoding.bronze.bronze_nasdaq_tickers
        ) AS ticker_combined
    """
)

unique_tickers

,ticker
0,AADI
1,ACCD
2,ACDC
3,ACET
4,ACHR
...,...
7128,KYTX
7129,MGX
7130,UMAC
7131,BF-B


In [5]:
current_df = bigquery.run_query(
            f"""
            SELECT
                ticker
            FROM {table_conca}
            """
            )
df_incremental = unique_tickers[~unique_tickers['ticker'].isin(current_df['ticker'])]
df_incremental

,ticker
2146,AKO/A
2147,AKO/B
2191,BF/A
2192,BF/B
2197,BIO/B
2206,BRK/A
2207,BRK/B
2241,CRD/A
2242,CRD/B
2322,HEI/A


In [13]:
import yfinance as yf
import pandas as pd
import os

# Cargar el progreso anterior si existe
if os.path.exists("ticker_data.csv"):
    df = pd.read_csv("ticker_data.csv", index_col=0)
    processed_tickers = set(df['ticker'])
else:
    df = pd.DataFrame()
    processed_tickers = set()

data = []

for i, row in unique_tickers.iterrows():
    ticker = row['ticker']
    
    # Saltar si el ticker ya ha sido procesado
    if ticker in processed_tickers:
        continue
    
    try:
        # Obtener datos históricos
        ticker_data = yf.Ticker(ticker).info
        
        # Convertir el diccionario en una fila de DataFrame
        data_row = pd.Series(ticker_data)
        
        # Agregar el símbolo como una columna en los datos históricos
        data_row['ticker'] = ticker
        
        # Añadir los datos a la lista
        data.append(data_row)
        
        # Guardar el progreso actual
        temp_df = pd.DataFrame(data)
        df = pd.concat([df, temp_df], ignore_index=True)
        df.to_csv("ticker_data.csv")
        
        # Limpiar la lista para la siguiente iteración
        data = []

    except Exception as e:
        print(f"Error procesando {ticker}: {e}")
        continue

# Si se completó, muestra los primeros registros
df.head()


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AGTI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AGTI&crumb=6ZoA%2FKE3xns
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/FSR?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=FSR&crumb=6ZoA%2FKE3xns
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/GRPH?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=GRPH&crumb=6ZoA%2FKE3xns
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ICVX?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.

Error procesando KOP: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out.


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/TAST?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=TAST&crumb=6ZoA%2FKE3xns
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MDC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MDC&crumb=6ZoA%2FKE3xns
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BRP?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=BRP&crumb=6ZoA%2FKE3xns
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/OCN?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yah

Error procesando AKO/A: Failed to parse json response from Yahoo Finance: {'code': 'Not Found', 'description': 'HTTP 404 Not Found'}


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AKO/B?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AKO%2FB&crumb=6ZoA%2FKE3xns


...

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/HFRO%5EA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=HFRO%5EA&crumb=6ZoA%2FKE3xns
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/HIG%5EG?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=HIG%5EG&crumb=6ZoA%2FKE3xns
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/HL%5EB?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=HL%5EB&crumb=6ZoA%2FKE3xns
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/HPP%5EC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDet

Error procesando HVT/A: Failed to parse json response from Yahoo Finance: {'code': 'Not Found', 'description': 'HTTP 404 Not Found'}


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/HWM%5E?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=HWM%5E&crumb=6ZoA%2FKE3xns
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ICR%5EA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ICR%5EA&crumb=6ZoA%2FKE3xns
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/IIPR%5EA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=IIPR%5EA&crumb=6ZoA%2FKE3xns
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/INN%5EE?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDet

Error procesando WSO/B: Failed to parse json response from Yahoo Finance: {'code': 'Not Found', 'description': 'HTTP 404 Not Found'}


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/XFLT%5EA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=XFLT%5EA&crumb=MsZqphFCtkk
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/YCBD%5EA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=YCBD%5EA&crumb=MsZqphFCtkk


In [20]:
import yfinance as yf
import pandas as pd

data = []

for i, row in df_incremental.iterrows():
    ticker = row['ticker']
    
    try:
        # Obtener datos históricos
        ticker_data = yf.Ticker(ticker).info
        
        # Convertir el diccionario en una fila de DataFrame
        data_row = pd.Series(ticker_data)
        
        # Agregar el símbolo como una columna en los datos históricos
        data_row['ticker'] = ticker
        
        # Añadir los datos a la lista
        data.append(data_row)
    
    except Exception as e:
        print(f"Error procesando {ticker}: {e}")
        # Continúa con el siguiente ticker si ocurre un error
        continue

# Concatenar todas las filas y convertir en un DataFrame
df = pd.DataFrame(data)
df.head()


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AKO/A?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AKO%2FA&crumb=MsZqphFCtkk


Error procesando AKO/A: Failed to parse json response from Yahoo Finance: {'code': 'Not Found', 'description': 'HTTP 404 Not Found'}


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AKO/B?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AKO%2FB&crumb=MsZqphFCtkk


Error procesando AKO/B: Failed to parse json response from Yahoo Finance: {'code': 'Not Found', 'description': 'HTTP 404 Not Found'}


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AL%5EA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AL%5EA&crumb=MsZqphFCtkk
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ALB%5EA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ALB%5EA&crumb=MsZqphFCtkk
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ALL%5EB?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ALL%5EB&crumb=MsZqphFCtkk
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ALL%5EH?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&cors

...

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/HWM%5E?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=HWM%5E&crumb=MsZqphFCtkk
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ICR%5EA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ICR%5EA&crumb=MsZqphFCtkk
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/IIPR%5EA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=IIPR%5EA&crumb=MsZqphFCtkk
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/INN%5EE?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&co

Error procesando WSO/B: Failed to parse json response from Yahoo Finance: {'code': 'Not Found', 'description': 'HTTP 404 Not Found'}


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/XFLT%5EA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=XFLT%5EA&crumb=MsZqphFCtkk
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/YCBD%5EA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=YCBD%5EA&crumb=MsZqphFCtkk


Error procesando None: 'NoneType' object has no attribute 'upper'


,trailingPegRatio,ticker,address1,address2,city,state,zip,country,phone,website,...,navPrice,category,ytdReturn,beta3Year,fundFamily,fundInceptionDate,legalType,threeYearAverageReturn,fiveYearAverageReturn,industrySymbol
0,NaN,AL^A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,ALB^A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,ALL^B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,ALL^H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,ALL^I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Concatenar todas las filas y convertir en un DataFrame
df = pd.DataFrame(data)
df

,trailingPegRatio,ticker,address1,address2,city,state,zip,country,phone,website,...,navPrice,category,ytdReturn,beta3Year,fundFamily,fundInceptionDate,legalType,threeYearAverageReturn,fiveYearAverageReturn,industrySymbol
0,NaN,AL^A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,ALB^A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,ALL^B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,ALL^H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,ALL^I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4969,NaN,KYTX,5980 Horton Street,Suite 550,Emeryville,CA,94608,United States,(510) 925-2492,https://kyvernatx.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4970,NaN,MGX,5959 Horton Street,7th Floor,Emeryville,CA,94608,United States,510-871-4880,https://www.metagenomi.co,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4971,NaN,UMAC,4667 1 B McLeod Rd,Suite J,Orlando,FL,32811,United States,855 921 4600,https://www.unusualmachines.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4972,4.6431,BF-B,850 Dixie Highway,NaN,Louisville,KY,40210,United States,502 585 1100,https://www.brown-forman.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# Identificar columnas de tipo 'object'
object_columns = df.select_dtypes(include=['object']).columns

# Convertir las columnas de tipo 'object' a listas
for col in object_columns:
    df[col] = df[col].apply(lambda x: json.dumps(x) if isinstance(x, (list, dict)) else x)

In [60]:
# Lista de columnas a eliminar
# columns_to_drop = ['priceToSalesTrailing12Months']
columns_to_drop = ['forwardPE', 'priceToSalesTrailing12Months']
columns_to_drop = ['industrySymbol']

# Eliminar las columnas
df = df.drop(columns=columns_to_drop)

In [24]:
df["trailingPE"] = df["trailingPE"].astype(str)

In [61]:
# Guardamos los datos en bigquery
bigquery.save_dataframe(df, project, dataset, table, if_exists='append', schema=None)

100%|██████████| 1/1 [00:00<?, ?it/s]
